## Importing LIbraries

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn  as sns
import numpy as np

import warnings

warnings.filterwarnings("ignore")


In [0]:
# Code to read csv file into Colaboratory:

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = "https://drive.google.com/open?id=19QIBAWLiehuZhV84eXTi2Mw5nR34hqIN"

In [0]:
fluff , id = link.split('=')

## Reading data

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('h1bvisa.csv')  
dataset = pd.read_csv('h1bvisa.csv', encoding ='latin1')

## Exploratory analysis

In [6]:
# samples out dataset
dataset.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067,20£1*6,"ANN ARBOR, MICHIGAN",-83.7430378,42.2808256
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES232,CHIEF OPERATING OFFICER,Y,242674,20£1*6,"PLANO, TEXAS",-96.6988856,33.0198431
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES232,CHIEF PROCESS OFFICER,Y,193066,20£1*6,"JERSEY CITY, NEW JERSEY",-74.0776417,40.7281575
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES232,"REGIONAL PRESIDEN, AMERICAS",Y,220314,20£1*6,"DENVER, COLORADO",-104.990251,39.7392358
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES232,PRESIDENT MONGOLIA AND INDIA,Y,157518.4,20£1*6,"ST. LOUIS, MISSOURI",-90.1994042,38.6270025


In [7]:
# checking data information
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002458 entries, 0 to 3002457
Data columns (total 11 columns):
Unnamed: 0            object
CASE_STATUS           object
EMPLOYER_NAME         object
SOC_NAME              object
JOB_TITLE             object
FULL_TIME_POSITION    object
PREVAILING_WAGE       object
YEAR                  object
WORKSITE              object
lon                   object
lat                   object
dtypes: object(11)
memory usage: 252.0+ MB


In [8]:
dataset.columns.tolist()

['Unnamed: 0',
 'CASE_STATUS',
 'EMPLOYER_NAME',
 'SOC_NAME',
 'JOB_TITLE',
 'FULL_TIME_POSITION',
 'PREVAILING_WAGE',
 'YEAR',
 'WORKSITE',
 'lon',
 'lat']

In [0]:
# feature selection
df = dataset.drop(columns=['Unnamed: 0','YEAR', 'lon','lat','JOB_TITLE'], index=1)

In [10]:
#Checking for nulls 

df.isnull().sum()

CASE_STATUS              13
EMPLOYER_NAME            59
SOC_NAME              17734
FULL_TIME_POSITION       15
PREVAILING_WAGE          85
WORKSITE                  0
dtype: int64

## Data wraggling

In [0]:
# dropping null obseravation on case status

df = df.dropna(axis= 0 , how='any', subset=['CASE_STATUS'])

In [0]:
#filling null values with mode
df['EMPLOYER_NAME'] = df['EMPLOYER_NAME'].fillna(df['EMPLOYER_NAME'].mode()[0])

In [0]:
df['FULL_TIME_POSITION'] = df['FULL_TIME_POSITION'].fillna(df['FULL_TIME_POSITION'].mode()[0])

In [14]:
#checking for unique values in position
df['FULL_TIME_POSITION'].value_counts()

Y     2576112
N      425920
No        412
Name: FULL_TIME_POSITION, dtype: int64

In [0]:
# replacing No with N
df['FULL_TIME_POSITION'] = df['FULL_TIME_POSITION'].replace('No', 'N')

In [0]:
# Changing wage dtype to float

df['PREVAILING_WAGE'] = pd.to_numeric(df['PREVAILING_WAGE'], errors='coerce')

In [0]:
# filling null with median

df['PREVAILING_WAGE'] = df['PREVAILING_WAGE'].fillna(df['PREVAILING_WAGE'].median())

In [18]:
df[df['SOC_NAME'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17721 entries, 17398 to 3002358
Data columns (total 6 columns):
CASE_STATUS           17721 non-null object
EMPLOYER_NAME         17721 non-null object
SOC_NAME              0 non-null object
FULL_TIME_POSITION    17721 non-null object
PREVAILING_WAGE       17721 non-null float64
WORKSITE              17721 non-null object
dtypes: float64(1), object(5)
memory usage: 969.1+ KB


In [0]:
#filling soc name null values with mode

df['SOC_NAME'] = df['SOC_NAME'].fillna(df['SOC_NAME'].mode()[0])

In [20]:
#checking unique values in Case status

df['CASE_STATUS'].value_counts()

CERTIFIED                                             2615623
CERTIFIED-WITHDRAWN                                    201707
DENIED                                                  94346
WITHDRAWN                                               89799
CERTIFIED-+--WITHDRAWN                                    951
PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED         15
REJECTED                                                    2
INVALIDATED                                                 1
Name: CASE_STATUS, dtype: int64

In [0]:
# dropping observation  with null as status case

df = df.drop(df[df['CASE_STATUS'] == 'WITHDRAWN'].index )
  

In [0]:
# function that converts status to 1 for Certfified and 0 for rejected 

def clean_status(val):
  if val == 'CERTIFIED':
    return 1
  else:
    return 0

In [0]:
df['CASE_STATUS'] = df['CASE_STATUS'].apply(clean_status)

In [24]:
# confirming new values in case status 
df['CASE_STATUS'].value_counts()

1    2615623
0     297022
Name: CASE_STATUS, dtype: int64

In [0]:
#Getting state from workste

df['WORKSITE'] = df['WORKSITE'].str.split('\s+').str[-1]

In [0]:
df['OCCUPATION'] = np.nan
df['SOC_NAME'] = df['SOC_NAME'].str.lower()
df.OCCUPATION[df['SOC_NAME'].str.contains('computer','programmer')] = 'computer occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('software','web developer')] = 'computer occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('database')] = 'computer occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('math','statistic')] = 'Mathematical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('predictive model','stats')] = 'Mathematical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('teacher','linguist')] = 'Education Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('professor','Teach')] = 'Education Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('school principal')] = 'Education Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('medical','doctor')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('physician','dentist')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('Health','Physical Therapists')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('surgeon','nurse')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('psychiatr')] = 'Medical Occupations'
df.OCCUPATION[df['SOC_NAME'].str.contains('chemist','physicist')] = 'Advance Sciences'
df.OCCUPATION[df['SOC_NAME'].str.contains('biology','scientist')] = 'Advance Sciences'
df.OCCUPATION[df['SOC_NAME'].str.contains('biologi','clinical research')] = 'Advance Sciences'
df.OCCUPATION[df['SOC_NAME'].str.contains('public relation','manage')] = 'Management Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('management','operation')] = 'Management Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('chief','plan')] = 'Management Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('executive')] = 'Management Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('advertis','marketing')] = 'Marketing Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('promotion','market research')] = 'Marketing Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('business','business analyst')] = 'Business Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('business systems analyst')] = 'Business Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('accountant','finance')] = 'Financial Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('financial')] = 'Financial Occupation'
df.OCCUPATION[df['SOC_NAME'].str.contains('engineer','architect')] = 'Architecture & Engineering'
df.OCCUPATION[df['SOC_NAME'].str.contains('surveyor','carto')] = 'Architecture & Engineering'
df.OCCUPATION[df['SOC_NAME'].str.contains('technician','drafter')] = 'Architecture & Engineering'
df.OCCUPATION[df['SOC_NAME'].str.contains('information security','information tech')] = 'Architecture & Engineering'
df['OCCUPATION']= df.OCCUPATION.replace(np.nan, 'Others', regex=True)

In [0]:
df['EMPLOYER_NAME'] = df['EMPLOYER_NAME'].str.lower()
df['EMPLOYER'] = np.nan
df['EMPLOYER'][df['EMPLOYER_NAME'].str.contains('university')]= 'university'
df['EMPLOYER']= df['EMPLOYER'].replace(np.nan, 'non university', regex=True)

In [0]:
df = df.drop(columns=['EMPLOYER_NAME','SOC_NAME'], axis=1)

## Feature selection

In [0]:
X = df[['FULL_TIME_POSITION','PREVAILING_WAGE','WORKSITE','OCCUPATION']]

In [0]:
y = df['CASE_STATUS'].values

## Test train

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

## Label encoding

In [0]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

In [0]:
X_train = pd.get_dummies(X_train)

In [0]:
X_test = pd.get_dummies(X_test)

## Train and predict

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [0]:
# KNN

from sklearn.neighbors import  KNeighborsClassifier

In [0]:
confus_matrix = []
# k_values = []
acc_score= []


# for k in  range(4,30):
knn = KNeighborsClassifier(n_neighbors= 15, p=1)
knn.fit(X_train, y_train)
knn_y_pred = knn.predict(X_test)
# k_values.append(k)
confus_matrix.append(confusion_matrix(y_test, knn_y_pred))
acc_score.append(accuracy_score(y_test, knn_y_pred))


## Evaluating a model 

### confusion matrix using 

In [39]:
confus_matrix

[array([[  3998,  84998],
        [  2486, 782312]])]

## accuracy score

In [40]:
acc_score

[0.8998802921512393]

## Report summary

In [0]:
from sklearn.metrics import classification_report

In [42]:
print(classification_report(y_test, knn_y_pred))

              precision    recall  f1-score   support

           0       0.62      0.04      0.08     88996
           1       0.90      1.00      0.95    784798

   micro avg       0.90      0.90      0.90    873794
   macro avg       0.76      0.52      0.52    873794
weighted avg       0.87      0.90      0.86    873794



## using random forest

In [0]:
from sklearn.ensemble import  RandomForestClassifier



randomforest = RandomForestClassifier(n_estimators=40 , random_state=0)



In [0]:
randomforest.fit(X_train, y_train)


rand_y_pred = randomforest.predict(X_test)

## Evaluating the model

### confusion matrix

In [45]:
confusion_matrix(y_test, rand_y_pred)

array([[  8842,  80154],
       [ 18289, 766509]])

### accuracy score 

In [46]:
confusion_matrix(y_test, rand_y_pred)

array([[  8842,  80154],
       [ 18289, 766509]])

### classification report

In [47]:
print(classification_report(y_test, rand_y_pred))

              precision    recall  f1-score   support

           0       0.33      0.10      0.15     88996
           1       0.91      0.98      0.94    784798

   micro avg       0.89      0.89      0.89    873794
   macro avg       0.62      0.54      0.55    873794
weighted avg       0.85      0.89      0.86    873794



## Questions

**Model Selection**

**Which model did you select and why?**

KNN because as compared with other models such as SVM , Decision Tree and Random Forest it performed a little bit better though not soo good especially in perdicting True Negative 

**How does the model work (Without Scikit-learn 😃 )**

Given a number n as neighbores , it counts which class majoriry obeservation falls into and classify the observation as part of that class.\

**Model instantiation**

**Are there any parameters you needed to instantiate for your model?**

Yes , number of neighbors and P value that determines with method to be used to identify distance between two point.

example
    default = 2 , for the Minkowski metric. When p = 1, this is
    equivalent to using manhattan distance (l1), and euclidean distance

**Describe two testing approaches in machine learning.**

**Training Dataset**
Prepare your model on the entire training dataset, then evaluate the model on the same dataset.


**Supplied Test Set**
Split your dataset manually using another program. Prepare your model on the entire training dataset and use the separate test set to evaluate the performance of the model. This is a good approach if you have a large dataset.

**Percentage Split**
Randomly split your dataset into a training and a testing partitions each time you evaluate a model. This can give you a very quick estimate of performance and like using a supplied test set, is preferable only when you have a large dataset.

**Cross Validation**
Split the dataset into k-partitions or folds. Train a model on all of the partitions except one that is held out as the test set, then repeat this process creating k-different models and give each fold a chance of being held out as the test set. Then calculate the average performance of all k models.

**In which situations would using accuracy to measure performance of a model a bad idea?**


Confusion matrix is the number of correct predictions out of all predictions made, often presented as a percentage where 100% is the best an algorithm can achieve. Can be misleading when used on unbalanced classes,

**Describe two other performance measures and when best to apply them.**

1. Confusion matrix. A table showing the number of predictions for each class compared to the number of instances that actually belong to each class. 

    useful when you want to compare how the model predicts correctly and wrongly.
    
2. Precision . Precision is the ratio of correct predictions to the total no. of predicted correct predictions. This measures how precise the classifier is when predicting positive instances.


**in the dataset above, which performance measures are most relevant.**

confusion matrix , 
Precision 
Recall